In [65]:
from os_helper import OsHelper
os_helper = OsHelper()

In [66]:
import pandas as pd
def use_base_data_instead_of_bloomberg(data: pd.DataFrame, name: str, max_year: int = 2019) -> pd.DataFrame:
    """
    This function formats the data from the base data folder to the same format as the Bloomberg data
    and filters out data beyond the year 2018, then converts the index back to a string format.

    Args:
        data (pd.DataFrame): Data to format correctly.
        name (str): Name of the column to be used as prefix for the ticker names.

    Returns:
        pd.DataFrame: DataFrame containing well formatted data up to the year 2018.
    """

    # Enlever le préfixe "{name}_" des noms des colonnes
    data.columns = data.columns.str.replace(f'{name}_', '')
    # Ajouter la chaîne " US Equity" à chaque nom de ticker restant
    data.columns = [f"{ticker} US Equity" for ticker in data.columns]
    # Changer le nom de la colonne 'date' en index et supprimer le nom de l'index
    data.index.rename(None, inplace=True)
    # Trier l'index dans l'ordre croissant
    data.sort_index(inplace=True)
    # Convertir l'index en DateTimeIndex pour faciliter le filtrage par année
    data.index = pd.to_datetime(data.index)
    # Conserver uniquement les données jusqu'à la fin max_year
    data = data[data.index.year <= max_year]
    # Convertir l'index de DateTimeIndex en chaîne de caractères au format 'YYYY-MM-DD'
    data.index = data.index.strftime('%Y-%m-%d')
    
    return data


In [67]:
close_data = os_helper.read_data(directory_name="base data", file_name="close.csv", index_col=0, low_memory=False)
volume_data = os_helper.read_data(directory_name="base data", file_name="volume.csv", index_col=0, low_memory=False)
adjusted_close_data = os_helper.read_data(directory_name="base data", file_name="adjusted_close.csv", index_col=0, low_memory=False)
print(close_data.head())
close = use_base_data_instead_of_bloomberg(data=close_data, name="close")
volume = use_base_data_instead_of_bloomberg(data=volume_data, name="volume")
adj_close = use_base_data_instead_of_bloomberg(data=adjusted_close_data, name="adjusted_close")

# save data
os_helper.write_data(directory_name="base data", file_name="new_close.csv", data_frame=close)
os_helper.write_data(directory_name="base data", file_name="new_volume.csv", data_frame=volume)
os_helper.write_data(directory_name="base data", file_name="new_tot_ret.csv", data_frame=adj_close)

Loaded file close.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/base data.
Loaded file volume.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/base data.
Loaded file adjusted_close.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/base data.
            close_CSCO  close_ISRG  close_TPR  close_DVN  close_MRO  close_BA  close_GILD  close_MDT  close_V  close_A  close_MO  close_CTRA  close_MCHP  close_EIX  close_BBY  close_GEN  close_WBA  close_DTE  close_C  close_T  ...  close_WYNN  close_RHI  close_EFX  close_NUE  close_CTSH  close_SWK  close_MU  close_TRV  close_L  close_AEP  close_CI  close_BALL  close_JNJ  close_DOV  close_CRM  close_PGR  close_WAT  close_BWA  close_BLK  close_PPL
timestamp                                                                                                                                 

In [68]:
from compute_metrics import ComputeMetrics

price_data = os_helper.read_data(directory_name="base data", file_name="new_close.csv", index_col=0, low_memory=False)
trade_volume = os_helper.read_data(directory_name="base data", file_name="new_volume.csv", index_col=0, low_memory=False)
spy = os_helper.read_data(directory_name="benchmark", file_name="SPY (1).csv", index_col=0, low_memory=False)

print(price_data.head())
print(trade_volume.head())
print(spy.head())

rolling_window_years = [1, 2, 3]

# Compute all risk metrics needed for the strategy and the filter
metrics_calculator = ComputeMetrics(price_data=price_data, trade_volume=trade_volume, rolling_window_years=rolling_window_years,
                                    benchmark_series=spy['Adj Close'])

final_df = metrics_calculator.compile_all_metrics()

os_helper.write_data(directory_name="transform data", file_name="all_metrics.csv", data_frame=final_df)

Loaded file new_close.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/base data.
Loaded file new_volume.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/base data.
Loaded file SPY (1).csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/benchmark.
            CSCO US Equity  ISRG US Equity  TPR US Equity  DVN US Equity  MRO US Equity  BA US Equity  GILD US Equity  MDT US Equity  V US Equity  A US Equity  MO US Equity  CTRA US Equity  MCHP US Equity  EIX US Equity  BBY US Equity  GEN US Equity  WBA US Equity  DTE US Equity  C US Equity  T US Equity  ...  WYNN US Equity  RHI US Equity  EFX US Equity  NUE US Equity  CTSH US Equity  SWK US Equity  MU US Equity  TRV US Equity  L US Equity  AEP US Equity  CI US Equity  BALL US Equity  JNJ US Equity  DOV US Equity  CRM US Equity  PGR US Equity  WAT US Equity  BWA US Equity  BLK US

In [69]:
from get_rebal_dates import GetRebalDates, filter_by_rebalance_dates

all_metrics = os_helper.read_data(directory_name="transform data", file_name="all_metrics.csv", index_col=0, header=[0, 1])
print(all_metrics.head())

termination_date = '2019-06-30' # Last day of backtest
initial_year = 1999 # First year of the backtest
reb_month = 12
reb_week = 3
reb_weekday = 'FRI'
reb_frequency = 'S' # Q for quarterly, M for monthly, A for annual, S for semi-annual

# Get the rebalance dates of the strategy
get_reb_dates = GetRebalDates(termination_date=termination_date, initial_year=initial_year, reb_month=reb_month,
                              reb_week=reb_week, reb_weekday=reb_weekday, reb_frequency=reb_frequency)
rebalance_dates = get_reb_dates.reb_dates

print("Dates de rebalancement :")
for date in rebalance_dates:
    print(date)

reb_metrics = filter_by_rebalance_dates(get_reb_dates=get_reb_dates, data=all_metrics)
print(reb_metrics.head())

os_helper.write_data(directory_name="transform data", file_name=f"rebalance_metrics.csv", data_frame=reb_metrics)

Loaded file all_metrics.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/transform data.
                   A US Equity                                                                                                                AAPL US Equity                                                                                                                 ABT US Equity                      ... YUM US Equity                     ZBH US Equity                                                                                                                ZION US Equity                                                                                                          
           average_turnover_1Y average_turnover_2Y average_turnover_3Y beta_1Y beta_2Y beta_3Y volatility_1Y volatility_2Y volatility_3Y average_turnover_1Y average_turnover_2Y average_turnover_3Y beta_1Y beta_2Y beta_3Y volatility_1Y volatility_2Y volatility_3Y aver

In [70]:
from filter import Filter

data = os_helper.read_data(directory_name="transform data", file_name="rebalance_metrics.csv", index_col=0)
print(data)
risk_filter = Filter(data=data)

risk_top_n = 140
liquidity_top_n = 200

filtered_data = risk_filter.apply_filters(liquidity_top_n=liquidity_top_n, risk_top_n=risk_top_n)
print(filtered_data.head())

os_helper.write_data(directory_name="transform data", file_name="filtered_data.csv", data_frame=filtered_data)

Loaded file rebalance_metrics.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/transform data.
             Date          Ticker  average_turnover_1Y  average_turnover_2Y  average_turnover_3Y   beta_1Y   beta_2Y   beta_3Y  volatility_1Y  volatility_2Y  volatility_3Y
0      2000-12-15     A US Equity         2.000540e+08                  NaN                  NaN  1.285401       NaN       NaN       1.082652            NaN            NaN
1      2000-12-15  AAPL US Equity         3.560394e+08                  NaN                  NaN  1.581072       NaN       NaN       0.977798            NaN            NaN
2      2000-12-15   ABT US Equity         3.427124e+08                  NaN                  NaN  0.173058       NaN       NaN       0.368400            NaN            NaN
3      2000-12-15  ADBE US Equity         2.422944e+08                  NaN                  NaN  2.246442       NaN       NaN       1.003055            NaN     

In [71]:
from weighting import InverseMetricsWeighting

filtered_data = os_helper.read_data(directory_name="transform data", file_name="filtered_data.csv", index_col=0)
print(filtered_data)

years_to_inverse = [1, 2, 3]
metric_to_inverse = "beta"
weighting_list = [] # Equal weighting of betas
min_weight = 0.00001
max_weight = 0.05

inverse_metrics_weighting = InverseMetricsWeighting(filtered_data=filtered_data, years_list=years_to_inverse,
                                                    target_metric=metric_to_inverse, weight_list=weighting_list,
                                                    min_limit=min_weight, max_limit=max_weight)
inverse_metrics_weighting_df = inverse_metrics_weighting.compute_inverse_weighted_metrics()
print(inverse_metrics_weighting_df)

os_helper.write_data(directory_name="transform data", file_name="inverse_metrics_weighting.csv", data_frame=inverse_metrics_weighting_df)

Loaded file filtered_data.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/transform data.
            Date          Ticker  average_turnover_1Y  average_turnover_2Y  average_turnover_3Y   beta_1Y   beta_2Y   beta_3Y  volatility_1Y  volatility_2Y  volatility_3Y
0     2002-12-13   ADM US Equity         1.928895e+07         2.096770e+07         1.995469e+07  0.354951  0.380743  0.367312       0.265237       0.308633       0.330793
1     2002-12-13   ADP US Equity         1.290485e+08         1.229771e+08         1.184466e+08  0.988513  0.885232  0.765565       0.421196       0.375484       0.377369
2     2002-12-13   AEE US Equity         2.740266e+07         2.205358e+07         1.965659e+07  0.486260  0.314623  0.289219       0.247957       0.233742       0.238914
3     2002-12-13   AEP US Equity         7.461715e+07         5.969405e+07         4.843943e+07  1.021821  0.639904  0.500216       0.573373       0.443004       0.40487

In [72]:
from portfolio_returns import PortfolioReturns

prices = os_helper.read_data(directory_name="base data", file_name="new_tot_ret.csv", index_col=0, sep=',')
print(prices.head())
weighting_df = os_helper.read_data(directory_name="transform data", file_name="inverse_metrics_weighting.csv", index_col=0)
print(weighting_df.head())

rebalance_lag = 7 # delay between weight computation and implementation
transaction_fee = 0.001 # 0.1% transaction fee

pf_returns = PortfolioReturns(prices=prices, weighting_data=weighting_df, rebalance_lag=rebalance_lag, transaction_fee=transaction_fee)
drifted_weights = pf_returns.drifted_weights
print(drifted_weights.head())
portfolio_returns = pf_returns.portfolio_returns
print(portfolio_returns.head())

os_helper.write_data(directory_name="final data", file_name="portfolio_returns.csv", data_frame=portfolio_returns)
os_helper.write_data(directory_name="final data", file_name="drifted_weights.csv", data_frame=drifted_weights)

Loaded file new_tot_ret.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/base data.
            CSCO US Equity  ISRG US Equity  TPR US Equity  DVN US Equity  MRO US Equity  BA US Equity  GILD US Equity  MDT US Equity  V US Equity  A US Equity  MO US Equity  CTRA US Equity  MCHP US Equity  EIX US Equity  BBY US Equity  GEN US Equity  WBA US Equity  DTE US Equity  C US Equity  T US Equity  ...  WYNN US Equity  RHI US Equity  EFX US Equity  NUE US Equity  CTSH US Equity  SWK US Equity  MU US Equity  TRV US Equity  L US Equity  AEP US Equity  CI US Equity  BALL US Equity  JNJ US Equity  DOV US Equity  CRM US Equity  PGR US Equity  WAT US Equity  BWA US Equity  BLK US Equity  PPL US Equity
1999-11-01       25.121693             NaN            NaN      12.686863       5.197426     28.661792        1.482176      23.654637          NaN          NaN      1.371226        1.034941        5.547514      14.141193      14.044962       2.290155 

In [73]:
from fama_regress import FamaFrenchRegression

ff_factors = os_helper.read_data(directory_name='benchmark', file_name='F-F_Research_Data_5_Factors_2x3_daily.csv')
portfolio_returns = os_helper.read_data(directory_name='final data', file_name='portfolio_returns.csv')

print(ff_factors.head())
print(portfolio_returns.head())

# Instantiate the FamaFrenchRegression class
ff_regression = FamaFrenchRegression(ff_factors, portfolio_returns)

# Run the regression
ff_regression.run_regression()

# Retrieve and display the regression results
regression_results = ff_regression.get_regression_results()
print(regression_results)

 # Run the rolling regression
rolling_results = ff_regression.run_rolling_regression(window_size=252)

# Print or use the rolling regression results
print(rolling_results)


Loaded file F-F_Research_Data_5_Factors_2x3_daily.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/benchmark.
Loaded file portfolio_returns.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/final data.
   Unnamed: 0  Mkt-RF   SMB   HML   RMW   CMA     RF
0    19630701   -0.67  0.02 -0.35  0.03  0.13  0.012
1    19630702    0.79 -0.28  0.28 -0.08 -0.21  0.012
2    19630703    0.63 -0.18 -0.10  0.13 -0.25  0.012
3    19630705    0.40  0.09 -0.28  0.07 -0.30  0.012
4    19630708   -0.63  0.07 -0.20 -0.27  0.06  0.012
   Unnamed: 0  Portfolio_Returns
0  2002-12-20           0.000000
1  2002-12-23           0.002887
2  2002-12-24          -0.003936
3  2002-12-26           0.004266
4  2002-12-27          -0.011185
                                 OLS Regression Results                                
Dep. Variable:          Excess_Return   R-squared (uncentered):                   0

In [81]:
from perf_report import PerfReport

benchmark_prices = os_helper.read_data(directory_name="benchmark", file_name="SPY (1).csv", index_col=0)
benchmark_prices.sort_index(inplace=True)
benchmark_prices.index.name = None
benchmark_prices = benchmark_prices[["Adj Close"]]
benchmark_prices.columns = ["SPY"]
# convert benchmark prices index to datetime
benchmark_prices.index = pd.to_datetime(benchmark_prices.index)
# keep only data from 2000-01-01 to 2014-12-31
benchmark_prices = benchmark_prices[benchmark_prices.index.year >= 2000]
# keep data before 2014-12-31 adn drop 
benchmark_prices = benchmark_prices[benchmark_prices.index.year <= 2014]
# convert benchmark prices index to string
benchmark_prices.index = benchmark_prices.index.strftime('%Y-%m-%d')

print(benchmark_prices.head())

strategy_returns = os_helper.read_data(directory_name="final data", file_name="portfolio_returns.csv", index_col=0)
strategy_returns.sort_index(inplace=True)
strategy_returns.index.name = None
# convert benchmark prices index to datetime
strategy_returns.index = pd.to_datetime(strategy_returns.index)
# keep only data from 2000-01-01 to 2014-12-31
strategy_returns = strategy_returns[strategy_returns.index.year >= 2000]
# keep data before 2014-12-31
strategy_returns = strategy_returns[strategy_returns.index.year <= 2014]
# convert benchmark prices index to string
strategy_returns.index = strategy_returns.index.strftime('%Y-%m-%d')

print(strategy_returns.head())
strategy_returns.columns = ["Portfolio_Returns"]

perf_report = PerfReport(port_returns=strategy_returns, bench_returns=benchmark_prices, strat_name="InverseMetrics")
print(perf_report.get_performance_indicators(rf=0.01, periods_per_year=252))
perf_report.generate_html_report(rf=0.01, periods_per_year=252)

Loaded file SPY (1).csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/benchmark.
                  SPY
2000-01-03  94.262543
2000-01-04  90.576317
2000-01-05  90.738373
2000-01-06  89.280067
2000-01-07  94.465073
Loaded file portfolio_returns.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/final data.
            Portfolio_Returns
2002-12-20           0.000000
2002-12-23           0.002887
2002-12-24          -0.003936
2002-12-26           0.004266
2002-12-27          -0.011185
{'Cumulative Returns': 4.814664083241165, 'CAGR': 0.1574637386137332, 'Sharpe': 0.9198111540309021, 'Sortino': 1.3165315008096155, 'Calmar': 0.36907310574680563, 'Max Drawdown': -0.4266464723705923, 'Average Return': 0.0006344252908167424, 'Volatility': 0.16293762735135794, 'Tail Ratio': 0.9892777123740502, 'Gain to Pain Ratio': 0.2037264220171056, 'Value at Risk (VaR)': -0.016248735786791677, 'Conditi

/Users/thomasvaudescal/.virtualenvs/PolyHacks/lib/python3.11/site-packages/quantstats/stats.py:968: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  returns = returns.pivot('Year', 'Month', 'Returns').fillna(0)


Moved file InverseMetrics_backtesting_report.html to folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Codes/reports.


In [82]:
from stickyRebalance import prepare_drifted_weights_format_submission

drifted_weights = os_helper.read_data(directory_name='final data', file_name='drifted_weights.csv', index_col=0)
print(drifted_weights.head())

submission_weights = prepare_drifted_weights_format_submission(drifted_weights=drifted_weights)

print(submission_weights.head())
print(submission_weights.tail())

os_helper.write_data(directory_name='final data', file_name='submission_weights.csv', data_frame=submission_weights)

Loaded file drifted_weights.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/final data.
            A US Equity  ACN US Equity  ADBE US Equity  ADI US Equity  ADM US Equity  ADP US Equity  ADSK US Equity  AEE US Equity  AEP US Equity  AES US Equity  AFL US Equity  AIZ US Equity  AKAM US Equity  ALL US Equity  AMAT US Equity  AMT US Equity  AON US Equity  APA US Equity  APD US Equity  APH US Equity  ...  TXT US Equity  UNP US Equity  UPS US Equity  VFC US Equity  VLO US Equity  VMC US Equity  VRSN US Equity  VTR US Equity  WAT US Equity  WBA US Equity  WDC US Equity  WEC US Equity  WELL US Equity  WMB US Equity  XEL US Equity  XRAY US Equity  XYL US Equity  YUM US Equity  ZBH US Equity  ZION US Equity
2002-12-20          0.0            0.0             0.0            0.0       0.007307       0.006088             0.0       0.007432       0.006365            0.0       0.006239            0.0             0.0       0.006611            

In [83]:
submission_weights.describe()

,weight_A,weight_ACN,weight_ADBE,weight_ADI,weight_ADM,weight_ADP,weight_ADSK,weight_AEE,weight_AEP,weight_AES,weight_AFL,weight_AIZ,weight_AKAM,weight_ALL,weight_AMAT,weight_AMT,weight_AON,weight_APA,weight_APD,weight_APH,...,weight_TXT,weight_UNP,weight_UPS,weight_VFC,weight_VLO,weight_VMC,weight_VRSN,weight_VTR,weight_WAT,weight_WBA,weight_WDC,weight_WEC,weight_WELL,weight_WMB,weight_XEL,weight_XRAY,weight_XYL,weight_YUM,weight_ZBH,weight_ZION
count,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,...,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000,4286.000000
mean,0.001149,0.002915,0.001304,0.002491,0.005985,0.006228,0.000901,0.009101,0.010373,0.003034,0.003838,0.003406,0.000135,0.005303,0.000415,0.004559,0.006279,0.001478,0.005228,0.002376,...,0.000418,0.001949,0.000196,0.006351,0.001398,0.001826,0.003795,0.007638,0.005558,0.002606,0.000320,0.010653,0.004476,0.000407,0.010086,0.006518,0.001026,0.004639,0.005516,0.002047
std,0.002286,0.003522,0.002196,0.002860,0.001994,0.001668,0.001965,0.002282,0.007818,0.002771,0.003096,0.002949,0.000780,0.002614,0.001340,0.003049,0.001912,0.003225,0.000669,0.002579,...,0.001365,0.002855,0.001136,0.000950,0.003557,0.002409,0.002949,0.004293,0.002366,0.003369,0.001287,0.003012,0.004868,0.001316,0.004768,0.001305,0.002063,0.003118,0.002592,0.002816
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004673,0.004839,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003069,0.000000,...,0.000000,0.000000,0.000000,0.003715,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006686,0.000000,0.000000,0.000000,0.003969,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.005098,0.005511,0.000000,0.007629,0.007038,0.000000,0.000000,0.000000,0.000000,0.005111,0.000000,0.000000,0.005303,0.000000,0.004841,0.000000,...,0.000000,0.000000,0.000000,0.005752,0.000000,0.000000,0.000000,0.005327,0.004632,0.000000,0.000000,0.009073,0.000000,0.000000,0.007906,0.005551,0.000000,0.000000,0.004997,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.005614,0.006385,0.000000,0.008754,0.008668,0.004530,0.005494,0.004965,0.000000,0.006089,0.000000,0.005763,0.006127,0.000000,0.005232,0.000000,...,0.000000,0.000000,0.000000,0.006352,0.000000,0.000000,0.005109,0.007325,0.005547,0.000000,0.000000,0.010038,0.005045,0.000000,0.009515,0.006481,0.000000,0.006127,0.006074,0.000000
75%,0.000000,0.006878,0.004233,0.005350,0.006584,0.007298,0.000000,0.009875,0.010022,0.005409,0.006235,0.005733,0.000000,0.007059,0.000000,0.007047,0.006893,0.000000,0.005706,0.004975,...,0.000000,0.005317,0.000000,0.007073,0.000000,0.004329,0.006203,0.011361,0.006961,0.006386,0.000000,0.010915,0.008562,0.000000,0.010707,0.007101,0.000000,0.007065,0.007166,0.005438
max,0.007014,0.009053,0.006304,0.008594,0.013761,0.010358,0.006363,0.016903,0.050441,0.008433,0.008846,0.009388,0.005445,0.008507,0.005291,0.008179,0.011089,0.010740,0.007082,0.007692,...,0.006036,0.007459,0.007099,0.008450,0.017257,0.007102,0.009266,0.017473,0.012772,0.009457,0.006644,0.022014,0.018288,0.006459,0.030019,0.010909,0.006771,0.008952,0.010173,0.007797
